<a href="https://colab.research.google.com/github/hemant-chapa23/ActivityDetector/blob/main/DL_Assignment_2_Working.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import *
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.utils import to_categorical
import pandas as pd
import numpy as np
from tqdm import tqdm
import os
import cv2
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, confusion_matrix
import itertools
import matplotlib.pyplot as plt
import shutil 

In [ ]:
base_path = '/content/drive/MyDrive/UCF-101/Dataset'
# images_path = '/content/drive/MyDrive/UCF-101/images/'
labels_path = '/content/drive/MyDrive/UCF-101-labels'
# os.makedirs("/content/drive/MyDrive/UCF-101/Dataset/train/", exist_ok = True)
# os.makedirs("/content/drive/MyDrive/UCF-101/Dataset/test/", exist_ok = True)
train_path = '/content/drive/MyDrive/UCF-101/Dataset/train/'
test_path = '/content/drive/MyDrive/UCF-101/Dataset/test/'
labels = {'BandMarching':0,'ApplyEyeMakeup':1,'Basketball':2,'BaseballPitch':3,'BabyCrawling':4}

In [ ]:
os.chdir(labels_path)

def make_train_test():
# print(os.getcwd())
  train_vdos = []
  test_vdos = []
  for category in os.listdir(labels_path):
    if "train" in category:
      train_file = open(category, 'r')
      for line in train_file:
        fields = line.split()
        if fields[0].split('_')[1] in labels:
          vdo = fields[0]
          # label = fields[1]
        # print(f"Video: {vdo}, Label: {label}")
          train_vdos.append((vdo, fields[0].split('_')[1]))
    elif "test" in category:
      test_file = open(category, 'r')
      for vdo in test_file:
        if vdo.split('_')[1] in labels:
          test_vdos.append(vdo) 

  # label_file = open('classInd.txt','r')
  # label_list = []
  # srl = []
  # classname = []
  # for line in label_file:
  #   fields = line.split()
  #   srl = fields[0]
  #   classname = fields[1]
  #   label_list.append((srl, classname))

  # labels_df = pd.DataFrame(label_list, columns = ["label", "class"])

  training = pd.DataFrame(train_vdos, columns = ["vdo", "label"])
  testing = pd.DataFrame(test_vdos, columns = ["vdo"])

  training.drop_duplicates(keep= 'first', inplace = True)
  testing.drop_duplicates(keep = 'first', inplace = True)
  # training = pd.merge(left = training, right = labels_df, how = 'inner', on = 'label')
  return training, testing


train, test = make_train_test()


In [ ]:
train.label.unique()
train

,vdo,label
0,ApplyEyeMakeup/v_ApplyEyeMakeup_g08_c01.avi,ApplyEyeMakeup
1,ApplyEyeMakeup/v_ApplyEyeMakeup_g08_c02.avi,ApplyEyeMakeup
2,ApplyEyeMakeup/v_ApplyEyeMakeup_g08_c03.avi,ApplyEyeMakeup
3,ApplyEyeMakeup/v_ApplyEyeMakeup_g08_c04.avi,ApplyEyeMakeup
4,ApplyEyeMakeup/v_ApplyEyeMakeup_g08_c05.avi,ApplyEyeMakeup
...,...,...
970,Basketball/v_Basketball_g06_c04.avi,Basketball
971,Basketball/v_Basketball_g07_c01.avi,Basketball
972,Basketball/v_Basketball_g07_c02.avi,Basketball
973,Basketball/v_Basketball_g07_c03.avi,Basketball


In [ ]:
# !rm -rf /content/drive/MyDrive/UCF-101

In [ ]:
def images_from_video(video_path):
  # print("images_from_video:", video_path)
  w = 224
  h = 224
  seq_len = 10
  frames = []
  vr = cv2.VideoCapture(video_path)

  frame_counts = int(vr.get(cv2.CAP_PROP_FRAME_COUNT))

  #Since extracting all the frames will be processing heavy, we create intervals on the frames that will be processed
  skip_frames = max(int(frame_counts/seq_len), 1)
  for part in range(seq_len): # This will generate 10frames per video
    vr.set(cv2.CAP_PROP_POS_FRAMES, skip_frames * part) # Capture the frame every skip_frames frames apart
    success, frame = vr.read()

    if not success:
      # print(success)
      break

    frame = cv2.resize(frame, (w, h))
    frame = frame/255
    frames.append(frame)

  vr.release()
  return frames


In [ ]:
"""
This will take only the video path and not the entire directory
 """
def load_videos(video_path):
  # print("load_videos:",video_path )
  images = []
  frames_list = images_from_video(video_path)
  images.append(frames_list)
  return np.array(images)

In [ ]:
""" Call to the load_videos"""
def get_frames_for_video(vdo_path):
  # print("get_frames_for_video: ", vdo_path)
  """
  This method accepts the number of videos to call load_vidoes for and get the frames for videos
  and returns a list of images with their frames and the training labels
  """
  # Call to load_videos
  img = load_videos(os.path.join(base_path,vdo_path)) 
  # image_list.append(img)
  training_label = train.loc[train.vdo == vdo_path,'label'].item()
  # print(training_label)
  return img, training_label

In [ ]:

def saving_train_images(image_list,train_label,video_list= []):
  os.chdir(train_path)
  """
  Plotting the Images with their training labels
  Each Video will generate 5 frames and each frame is plotted
  """
  # if save_or_display.lower() == 'display':
  #   fig, axes = plt.subplots(len(image_list), 10, figsize = (25,20))
  #   for (index, each_video) in enumerate(image_list):
  #     for (image_index, each_frame) in enumerate(each_video):
  #       for idx, frm in enumerate(each_frame):      
  #         axes[index][idx].imshow(frm, interpolation = 'nearest')
  #         axes[index][idx].set_title(train_label[index], fontdict={'fontsize': 15, 'fontweight': 'medium'})
  # elif save_or_display.lower() == 'save':
  for (index, each_video) in enumerate(image_list):
    for (image_index, each_frame) in enumerate(each_video):
      for idx, frm in enumerate(each_frame):      
        
        # fig = plt.figure()
        plt.imsave(f'{video_list[index]}_{index}_{idx}.jpg',frm)
        # plt.savefig(f'{video_list[index]}_{index}_{idx}.jpg')
        # plt.close(fig)

  return None

In [ ]:
# labels = {'BandMarching':0,'ApplyEyeMakeup':1,'Basketball':2,'BaseballPitch':3,'BabyCrawling':4}

In [ ]:
train = train[train['label'].isin(['ApplyEyeMakeup','BabyCrawling','Basketball','BaseballPitch','BandMarching'])]
train.label.unique()

array(['BandMarching', 'BaseballPitch', 'Basketball'], dtype=object)

In [ ]:
train[train['label']=='Basketball']

,vdo,label
417,Basketball/v_Basketball_g08_c01.avi,Basketball
418,Basketball/v_Basketball_g08_c02.avi,Basketball
419,Basketball/v_Basketball_g08_c03.avi,Basketball
420,Basketball/v_Basketball_g08_c04.avi,Basketball
421,Basketball/v_Basketball_g09_c01.avi,Basketball
...,...,...
970,Basketball/v_Basketball_g06_c04.avi,Basketball
971,Basketball/v_Basketball_g07_c01.avi,Basketball
972,Basketball/v_Basketball_g07_c02.avi,Basketball
973,Basketball/v_Basketball_g07_c03.avi,Basketball


In [ ]:
# train['label'].unique()
import gc
import time

In [ ]:
image_list = []
training_labels = []
vdo_list= []
epoch = 0 
for vdo in tqdm(train["vdo"]):
  image, label = get_frames_for_video(vdo)
  image_list.append(image)
  training_labels.append(label)
  vdo_list.append(vdo.split("/")[1].split(".")[0])
saving_train_images(image_list, training_labels, vdo_list) # Saved in train folder
  # epoch = epoch+1
  # if epoch%20==0:
  #   gc.collect()
  #   time.sleep(10)

# plot_images(image_list, training_labels, vdo_list, save_or_display = 'display') # display interactively

100%|██████████| 439/439 [03:35<00:00,  2.03it/s]


In [ ]:
len(os.listdir('/content/drive/MyDrive/UCF-101/Dataset/train'))

8329

In [ ]:
train['vdo'] = '/content/drive/MyDrive/UCF-101/Dataset/'+train['vdo']

In [ ]:
train

,vdo,label
198,/content/drive/MyDrive/UCF-101/Dataset/BandMar...,BandMarching
199,/content/drive/MyDrive/UCF-101/Dataset/BandMar...,BandMarching
200,/content/drive/MyDrive/UCF-101/Dataset/BandMar...,BandMarching
201,/content/drive/MyDrive/UCF-101/Dataset/BandMar...,BandMarching
202,/content/drive/MyDrive/UCF-101/Dataset/BandMar...,BandMarching
...,...,...
970,/content/drive/MyDrive/UCF-101/Dataset/Basketb...,Basketball
971,/content/drive/MyDrive/UCF-101/Dataset/Basketb...,Basketball
972,/content/drive/MyDrive/UCF-101/Dataset/Basketb...,Basketball
973,/content/drive/MyDrive/UCF-101/Dataset/Basketb...,Basketball
